In [1]:
#memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() #проверяем на засраность

Gen RAM Free: 12.8 GB  | Proc size: 138.1 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
 !kill -9 -1 #чистим гавно если надо (перезагузка collab)

In [2]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x576fa000 @  0x7f354d30b2a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [3]:
import torchvision.models as models
model = models.vgg16(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:07<00:00, 72243892.06it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

(fc): Linear(in_features=4096, out_features=1000, bias=True)

In [4]:
import torch.nn as nn
num_classes = 20
model.classifier[-1] = nn.Linear(in_features=4096, out_features=num_classes)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

(fc): Linear(in_features=4096, out_features=20, bias=True)

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [6]:
!ls -la /gdrive/My\ Drive/ML
!cp /gdrive/My\ Drive/ML/VOC2007.tar.gz .
!ls .
!tar -xvf VOC2007.tar.gz

total 2624515
-r-------- 2 root root  753575686 Oct 31 10:02  dogscats_all.tar.gz
-r-------- 2 root root  559404776 Oct 31 12:45  dogscats_train.tar.gz
-rw------- 1 root root 1374517542 Nov  5 16:55  VOC2007.tar.gz
dr-x------ 2 root root       4096 Oct 30 12:48 'Наборы данных'
sample_data  VOC2007.tar.gz
sample_data/VOC2007/VOC2007/
sample_data/VOC2007/VOC2007/test/
sample_data/VOC2007/VOC2007/test/cat/
sample_data/VOC2007/VOC2007/test/cat/000651.jpg
sample_data/VOC2007/VOC2007/test/cat/002003.jpg
sample_data/VOC2007/VOC2007/test/cat/007829.jpg
sample_data/VOC2007/VOC2007/test/cat/007441.jpg
sample_data/VOC2007/VOC2007/test/cat/000114.jpg
sample_data/VOC2007/VOC2007/test/cat/002552.jpg
sample_data/VOC2007/VOC2007/test/cat/009234.jpg
sample_data/VOC2007/VOC2007/test/cat/001389.jpg
sample_data/VOC2007/VOC2007/test/cat/004188.jpg
sample_data/VOC2007/VOC2007/test/cat/001173.jpg
sample_data/VOC2007/VOC2007/test/cat/007332.jpg
sample_data/VOC2007/VOC2007/test/cat/006743.jpg
sample_data/VOC20

In [0]:
from  torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import numpy as np
IMAGE_FOLDER_ROOT = 'sample_data/VOC2007/VOC2007'

In [0]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
data_train = ImageFolder(IMAGE_FOLDER_ROOT+'/trainval',transform=transforms.Compose([transforms.Resize(224),transforms.RandomCrop(224),transforms.ToTensor(),normalize]))
data_test = ImageFolder(IMAGE_FOLDER_ROOT+'/test',transform=transforms.Compose([transforms.Resize(224),transforms.RandomCrop(224),transforms.ToTensor(),normalize]))

In [20]:
train_loader = torch.utils.data.DataLoader(data_train, batch_size=16, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=16, shuffle=True, num_workers=8)
print(len(data_train), len(data_test))

7913 7632


In [10]:
!pip install Pillow==4.1.1
!pip install PIL
!pip install image
import PIL.Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

    100% |████████████████████████████████| 5.7MB 7.0MB/s 
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL
    100% |████████████████████████████████| 7.3MB 6.1MB/s 


In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
import torch.optim
import matplotlib.pyplot as plt
import numpy as np

def accuracy(output,labels):
  predictions = torch.argmax(output,dim=1)
  correct = (predictions == labels).sum().cpu().numpy()
  return correct / len(labels)

print(device)

cuda:0


In [0]:

# model.eval()

# accuracies = []
# with torch.no_grad():
#   for itr,x in enumerate(test_loader):
#     data = x[0]
#     labels = x[1]

#     data = data.to(device)
#     labels = labels.to(device)

#     y_pred = model.forward(data)
#     acc = accuracy(y_pred,labels)
#     accuracies.append(acc)
#     print(device,itr,'/',len(test_loader),acc)


# print('accuracy {:.2f}'.format(np.mean(accuracies)))

In [21]:
from torch.optim import lr_scheduler
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

lossf = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True)

# Decay LR by a factor of 0.1 every 7 epochs
#lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
model.to(device)
#model.train()


for epoch in range(3):
  for itr,x in enumerate(train_loader):
    data = x[0]
    labels = x[1]

    optimizer.zero_grad()

    data = data.to(device)
    labels = labels.to(device)
    y_pred = model.forward(data)

    loss = lossf(y_pred,labels)
    loss.backward()

    optimizer.step(closure)

    print('Iteration {}, train accuracy {:.2f}, loss {:.4f}'.format(itr+epoch*len(train_loader),accuracy(y_pred,labels),loss))

RuntimeError: ignored

In [22]:
printm()

Gen RAM Free: 11.7 GB  | Proc size: 2.1 GB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
model.eval()

accuracies = []
with torch.no_grad():
  for itr,x in enumerate(test_loader):
    data = x[0]
    labels = x[1]

    optimizer.zero_grad()

    data = data.to(device)
    labels = labels.to(device)

    y_pred = model.forward(data)
    accuracies.append(accuracy(y_pred,labels))
    

print('accuracy {:.2f}'.format(np.mean(accuracies)))